# Lambda = 4, lambda = 2.0: EvolvedOperatorAnsatz

In [12]:
import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import time

import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import EvolvedOperatorAnsatz

import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

# Load H

In [3]:
data_coeff, data_str = open_n_process("../utility/pauliH_L4_g2.0.txt")
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.008975
Max absolute value is 2.89778
Mean absolute value is 0.25249
None
E_exact = 3.89548


In [4]:
#get the indices of the N largest elements
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)
ind25, ops25, vals25 = get_k_largest_ops(data_coeff, data_str, 25)
ind30, ops30, vals30 = get_k_largest_ops(data_coeff, data_str, 30)
ind40, ops40, vals40 = get_k_largest_ops(data_coeff, data_str, 40)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_Hp25 =[Pauli(f'{ops25[i]}') for i in range(len(ops25))]
ops_Hp30 =[Pauli(f'{ops30[i]}') for i in range(len(ops30))]
ops_Hp40 =[Pauli(f'{ops40[i]}') for i in range(len(ops40))]

In [5]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_Hp25 = EvolvedOperatorAnsatz(ops_Hp25, reps=1, insert_barriers=True)
ev_op_Hp30 = EvolvedOperatorAnsatz(ops_Hp30, reps=1, insert_barriers=True)
ev_op_Hp40 = EvolvedOperatorAnsatz(ops_Hp40, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_Hp25_2f = EvolvedOperatorAnsatz(ops_Hp25, reps=2, insert_barriers=True)
ev_op_Hp30_2f = EvolvedOperatorAnsatz(ops_Hp30, reps=2, insert_barriers=True)
ev_op_Hp40_2f = EvolvedOperatorAnsatz(ops_Hp40, reps=2, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_Hp25, ev_op_Hp30, ev_op_Hp40, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_Hp25_2f, ev_op_Hp30_2f, ev_op_Hp40_2f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_Hp25', 'ev_op_Hp30', 'ev_op_H40',
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_Hp25_2f', 'ev_op_Hp30_2f', 'ev_op_Hp40_2f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 25, 30, 40, 30, 40, 50, 60, 80]


In [14]:
#[print(list(zip(ops30, vals30))[i]) for i in range(30)]

# QVE individual runs

In [6]:
seed = 170
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz, iters):
    opt = optimizer(maxiter = iters)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA

### 550 iterations:

At step 0, with ev_op_Hp20
VQE result: 4.88594
Length of this optimization 227, time taken = 296.544 

At step 1, with ev_op_Hp30
VQE result: 7.16187
Length of this optimization 392, time taken = 798.849 

At step 2, with ev_op_H40
VQE result: 6.05659
Length of this optimization 550, time taken = 1607.398 

At step 3, with ev_op_Hp20_2f
VQE result: 4.99013
Length of this optimization 430, time taken = 862.027 

At step 4, with ev_op_Hp30_2f
VQE result: 10.74866
Length of this optimization 550, time taken = 1784.653 

At step 5, with ev_op_Hp40_2f
VQE result: 21.88700
Length of this optimization 550, time taken = 2719.524 

In [7]:
r_cobyla=[]
iters = 650
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')   
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(COBYLA, ansatz_list[i], iters)
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 5.18872
Length of this optimization 205, time taken = 217.991 

At step 1, with ev_op_Hp20
VQE result: 4.48164
Length of this optimization 279, time taken = 302.564 

At step 2, with ev_op_Hp25
VQE result: 6.16766
Length of this optimization 324, time taken = 380.013 

At step 3, with ev_op_Hp30
VQE result: 4.88188
Length of this optimization 452, time taken = 655.43 

At step 4, with ev_op_H40
VQE result: 14.99081
Length of this optimization 441, time taken = 915.525 

At step 5, with ev_op_Hp15_2f
VQE result: 3.83330
Length of this optimization 323, time taken = 431.424 

At step 6, with ev_op_Hp20_2f
VQE result: 5.55726
Length of this optimization 429, time taken = 637.505 

At step 7, with ev_op_Hp25_2f
VQE result: 7.39414
Length of this optimization 582, time taken = 974.137 

At step 8, with ev_op_Hp30_2f
VQE result: 5.62945
Length of this optimization 650, time taken = 1414.989 

At step 9, with ev_op_Hp40_2f
VQE result: 19.78529
Length of 

In [8]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv('l4_l2.0_op_ev_20_30_40_indv_cobyla.csv')

# SPSA 
### 1000 iterations

At step 0, with ev_op_Hp20
VQE result: 5.07011
Length of this optimization 2051, time taken = 2018.96 

At step 1, with ev_op_Hp30
VQE result: 4.59715
Length of this optimization 2051, time taken = 2634.783 

At step 2, with ev_op_H40
VQE result: 4.52666
Length of this optimization 2051, time taken = 3522.86 

At step 3, with ev_op_Hp20_2f
VQE result: 4.67121
Length of this optimization 2051, time taken = 2750.232 

At step 4, with ev_op_Hp30_2f
VQE result: 5.05835
Length of this optimization 2051, time taken = 3919.943 

### 550 iterations

At step 0, with ev_op_Hp15
VQE result: 4.61577
Length of this optimization 1151, time taken = 1548.719 

At step 1, with ev_op_Hp20
VQE result: 4.72853
Length of this optimization 1151, time taken = 1670.138 

At step 2, with ev_op_Hp25
VQE result: 4.77875
Length of this optimization 1151, time taken = 6832.995 

At step 3, with ev_op_Hp30
VQE result: 4.12597
Length of this optimization 1151, time taken = 20321.974 

At step 4, with ev_op_H40
VQE result: 7.83938
Length of this optimization 1151, time taken = 2912.388 

At step 5, with ev_op_Hp15_2f
VQE result: 4.03526
Length of this optimization 1151, time taken = 1944.099 

At step 6, with ev_op_Hp20_2f
VQE result: 4.51575
Length of this optimization 1151, time taken = 2068.24

In [9]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i],650)
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    
print(len(r_spsa))
    

At step 0, with ev_op_Hp15
VQE result: 5.26024
Length of this optimization 1351, time taken = 1386.879 

At step 1, with ev_op_Hp20
VQE result: 4.82453
Length of this optimization 1351, time taken = 1465.449 

At step 2, with ev_op_Hp25
VQE result: 4.63808
Length of this optimization 1351, time taken = 1562.864 

At step 3, with ev_op_Hp30
VQE result: 4.26867
Length of this optimization 1351, time taken = 1871.368 

At step 4, with ev_op_H40
VQE result: 4.56014
Length of this optimization 1351, time taken = 2476.029 

At step 5, with ev_op_Hp15_2f
VQE result: 4.12574
Length of this optimization 1351, time taken = 1748.22 

At step 6, with ev_op_Hp20_2f
VQE result: 4.10240
Length of this optimization 1351, time taken = 1900.309 

At step 7, with ev_op_Hp25_2f
VQE result: 4.52611
Length of this optimization 1351, time taken = 2134.527 

At step 8, with ev_op_Hp30_2f
VQE result: 4.76864
Length of this optimization 1351, time taken = 2757.219 

At step 9, with ev_op_Hp40_2f
VQE result: 12.

In [10]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv('l4_l2.0_op_ev_20_30_40_indv_spsa.csv')

In [12]:
r_spsa2=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(SPSA, ansatz_list[i],850)
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa2.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    
print(len(r_spsa2))

At step 0, with ev_op_Hp15
VQE result: 4.95306
Length of this optimization 1751, time taken = 1431.4 

At step 1, with ev_op_Hp20
VQE result: 4.95798
Length of this optimization 1751, time taken = 1536.466 

At step 2, with ev_op_Hp25
VQE result: 4.64100
Length of this optimization 1751, time taken = 1674.356 

At step 3, with ev_op_Hp30
VQE result: 4.77990
Length of this optimization 1751, time taken = 2071.127 

At step 4, with ev_op_H40
VQE result: 4.44978
Length of this optimization 1751, time taken = 2823.295 

At step 5, with ev_op_Hp15_2f
VQE result: 4.62553
Length of this optimization 1751, time taken = 1925.163 

At step 6, with ev_op_Hp20_2f
VQE result: 4.32279
Length of this optimization 1751, time taken = 2096.367 

At step 7, with ev_op_Hp25_2f
VQE result: 4.62304
Length of this optimization 1751, time taken = 2423.337 

At step 8, with ev_op_Hp30_2f
VQE result: 4.85916
Length of this optimization 1751, time taken = 25381.44 

At step 9, with ev_op_Hp40_2f
VQE result: 15.4

In [13]:
df3 = pd.concat([r_spsa2[i] for i in range(len(r_spsa2))], axis = 1)
df3.to_csv('l4_l2.0_op_ev_20_30_40_indv_spsa_850.csv')